In [1]:
import backtrader as bt
import datetime
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
class IchimokuStrategy(bt.Strategy):
    params = (
        ('tenkan', 9),
        ('kijun', 26),
        ('senkou', 52),
        ('senkou_lead', 26),
        ('chikou', 26),
        ('sar_af', 0.02),
        ('sar_max', 0.2),
        ('stop_loss', 0.03),     # Stop loss %3'e düşürüldü
        ('take_profit', 0.06),   # Take profit %6'ya düşürüldü
        ('position_size', 0.2),
        ('rsi_period', 14),
        ('rsi_overbought', 70),
        ('rsi_oversold', 30)
    )
    
    def __init__(self):
        # Ichimoku göstergeleri
        self.ichi = bt.indicators.Ichimoku()
        self.tenkan_line = self.ichi.tenkan_sen
        self.kijun_line = self.ichi.kijun_sen
        self.senkou_a = self.ichi.senkou_span_a
        self.senkou_b = self.ichi.senkou_span_b
        self.chikou_line = self.ichi.chikou_span
        
        # SAR
        self.sar = bt.indicators.ParabolicSAR()
        
        # RSI
        self.rsi = bt.indicators.RSI(period=self.p.rsi_period)
        
        # Moving Averages
        self.sma20 = bt.indicators.SimpleMovingAverage(period=20)
        self.sma50 = bt.indicators.SimpleMovingAverage(period=50)
        
        # Kayıt tutma
        self.order = None
        self.stop_price = None
        self.target_price = None
        self.last_sell_price = None
        
        # Pozisyon ve değer takibi
        self.portfolio_value = []
        self.position_size = []

    def next(self):
        # Değerleri kaydet
        self.portfolio_value.append(self.broker.getvalue())
        self.position_size.append(self.position.size if self.position else 0)
        
        if self.order:
            return
        
        # Pozisyonda isek
        if self.position:
            # Stop loss kontrolü
            self.log(f'Current Close: {self.data.close[0]:.2f}')
            self.log(f'Stop Price: {self.stop_price:.2f}')
            self.log(f'Target Price: {self.target_price:.2f}')
            self.log(f'Tenkan vs Kijun: {self.tenkan_line[0]:.2f} vs {self.kijun_line[0]:.2f}')
            self.log(f'RSI: {self.rsi[0]:.2f}')
            self.log(f'SAR: {self.sar[0]:.2f}')    
            if self.data.close[0] <= self.stop_price:
                self.log(f'STOP LOSS SELL @ {self.data.close[0]:.2f}')
                self.order = self.sell()
                self.last_sell_price = self.data.close[0]
                return
            
            # Take profit kontrolü
            if self.data.close[0] >= self.target_price:
                self.log(f'TAKE PROFIT SELL @ {self.data.close[0]:.2f}')
                self.order = self.sell()
                self.last_sell_price = self.data.close[0]
                return
            
            # Trend değişimi satış sinyalleri
            sell_signals = [
                self.tenkan_line[0] < self.kijun_line[0],    # Tenkan Kijun crossover
                self.data.close[0] < self.senkou_a[0],       # Fiyat Kumo altında
                self.data.close[0] < self.sar[0],            # SAR altında
                self.rsi[0] > 65,                            # RSI yüksek (70'den 65'e düşürüldü)
                self.data.close[0] < self.data.close[-1]     # Fiyat düşüşte
            ]

            # Herhangi iki sinyal varsa sat
            if sum(sell_signals) >= 2:
                self.log(f'TREND CHANGE SELL @ {self.data.close[0]:.2f}')
                self.order = self.sell()
                self.last_sell_price = self.data.close[0]
                return
                    
        # Pozisyonda değilsek
        else:
            # Alım sinyalleri
            buy_signals = [
                self.tenkan_line[0] > self.kijun_line[0],    # Tenkan Kijun crossover
                self.data.close[0] > self.senkou_a[0],       # Fiyat Kumo üstünde
                self.data.close[0] > self.senkou_b[0],       # Fiyat Kumo üstünde
                self.data.close[0] > self.sma20[0],          # Fiyat 20 SMA üstünde
                self.sma20[0] > self.sma50[0],               # 20 SMA 50 SMA üstünde
                self.rsi[0] < self.p.rsi_overbought,         # RSI aşırı alım değil
                self.data.close[0] > self.sar[0]             # SAR üstünde
            ]
            
            # Geri çekilme alım sinyalleri
            pullback_signals = [
                self.data.close[0] > self.kijun_line[0],     # Fiyat Kijun üstünde
                self.rsi[0] < 40,                            # RSI düşük seviyede
                self.data.close[0] > self.data.close[-1],    # Yükseliş başlangıcı
                self.last_sell_price is None or              # İlk alım veya
                self.data.close[0] < self.last_sell_price    # Önceki satıştan düşük
            ]
            
            if sum(buy_signals) >= 5 or sum(pullback_signals) >= 3:  # Güçlü alım veya geri çekilme
                size = self.broker.getcash() * self.p.position_size / self.data.close[0]
                self.log(f'BUY CREATE @ {self.data.close[0]:.2f}')
                self.order = self.buy(size=size)
                
                # Stop loss ve take profit
                self.stop_price = self.data.close[0] * (1 - self.p.stop_loss)
                self.target_price = self.data.close[0] * (1 + self.p.take_profit)
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')

In [3]:
def calculate_indicators(data):
    """
    Teknik göstergeleri hesapla
    """
    # Ichimoku hesaplamaları
    high_9 = data['High'].rolling(window=9).max()
    low_9 = data['Low'].rolling(window=9).min()
    data['Tenkan_sen'] = (high_9 + low_9) / 2

    high_26 = data['High'].rolling(window=26).max()
    low_26 = data['Low'].rolling(window=26).min()
    data['Kijun_sen'] = (high_26 + low_26) / 2

    data['Senkou_Span_A'] = ((data['Tenkan_sen'] + data['Kijun_sen']) / 2).shift(26)

    high_52 = data['High'].rolling(window=52).max()
    low_52 = data['Low'].rolling(window=52).min()
    data['Senkou_Span_B'] = ((high_52 + low_52) / 2).shift(26)

    data['Chikou_Span'] = data['Close'].shift(-26)

    # RSI hesaplama
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # SAR hesaplama
    af = 0.01  # Başlangıç hızlanma faktörünü düşürdük
    max_af = 0.15  # Maksimum hızlanma faktörünü düşürdük
    trending_high = data['High'].iloc[0]
    trending_low = data['Low'].iloc[0]
    is_uptrend = True
    sar_values = []
    sar = trending_low

    for i in range(len(data)):
        if is_uptrend:
            sar = min(sar + af * (trending_high - sar), 
                     data['Low'].iloc[i-1] if i > 0 else data['Low'].iloc[0])
            if data['Low'].iloc[i] < sar:
                is_uptrend = False
                sar = trending_high
                af = 0.01
                trending_low = data['Low'].iloc[i]
            else:
                if data['High'].iloc[i] > trending_high:
                    trending_high = data['High'].iloc[i]
                    af = min(af + 0.01, max_af)
        else:
            sar = max(sar + af * (trending_low - sar), 
                     data['High'].iloc[i-1] if i > 0 else data['High'].iloc[0])
            if data['High'].iloc[i] > sar:
                is_uptrend = True
                sar = trending_low
                af = 0.01
                trending_high = data['High'].iloc[i]
            else:
                if data['Low'].iloc[i] < trending_low:
                    trending_low = data['Low'].iloc[i]
                    af = min(af + 0.01, max_af)
        
        sar_values.append(sar)

    data['SAR'] = sar_values

    # NaN değerleri temizle
    return data.dropna()

In [4]:
# Veri indirme fonksiyonu
def get_crypto_data(symbol="BTC-USD", start="2022-01-01", end=None):
    if end is None:
        end = datetime.datetime.now()
    df = yf.download(symbol, start=start, end=end)
    return df

In [5]:
def run_backtest(data, strategy=IchimokuStrategy, **kwargs):
    cerebro = bt.Cerebro()
    
    # Veriyi hazırla
    feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(feed)
    
    # Stratejiyi ekle
    cerebro.addstrategy(strategy, **kwargs)
    
    # Başlangıç parası
    initial_cash = 100000.0
    cerebro.broker.setcash(initial_cash)
    
    # Komisyon ve slippage
    cerebro.broker.setcommission(commission=0.001)  # %0.1 komisyon
    cerebro.broker.set_slippage_perc(0.001)        # %0.1 slippage
    
    # Risk yönetimi için position sizer
    cerebro.addsizer(bt.sizers.PercentSizer, percents=20)  # Her işlem için %20
    
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    # Stratejiyi çalıştır
    results = cerebro.run()
    strat = results[0]
    
    # Son değer
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % final_value)
    
    # Performans metrikleri
    roi = (final_value - initial_cash) / initial_cash * 100
    print(f'ROI: {roi:.2f}%')
    
    # Strateji sonuçlarını DataFrame'e ekle
    portfolio_values = [initial_cash] + strat.portfolio_value
    position_sizes = [0] + strat.position_size
    
    # Uzunlukları kontrol et ve ayarla
    data_len = len(data)
    if len(portfolio_values) > data_len:
        portfolio_values = portfolio_values[:data_len]
        position_sizes = position_sizes[:data_len]
    elif len(portfolio_values) < data_len:
        portfolio_values.extend([portfolio_values[-1]] * (data_len - len(portfolio_values)))
        position_sizes.extend([position_sizes[-1]] * (data_len - len(position_sizes)))
    
    # DataFrame'e ekle
    data = data.copy()  # Orijinal veriyi değiştirmemek için kopya oluştur
    data['Portfolio_Value'] = portfolio_values
    data['Position'] = position_sizes
    data['Signal'] = 0
    
    # Alım-satım sinyallerini işaretle
    for i in range(1, len(data)):
        if data['Position'].iloc[i] > 0 and data['Position'].iloc[i-1] == 0:
            data.loc[data.index[i], 'Signal'] = 1
        elif data['Position'].iloc[i] == 0 and data['Position'].iloc[i-1] > 0:
            data.loc[data.index[i], 'Signal'] = -1
    
    return data, results

In [6]:
def plot_strategy(data):
    fig = make_subplots(rows=4, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.05,
                        row_heights=[0.5, 0.2, 0.15, 0.15],
                        subplot_titles=('Ichimoku Cloud & SAR', 'Portfolio Value', 'RSI', 'Positions'))

    # Ana grafik - Mum grafiği
    fig.add_trace(go.Candlestick(x=data.index,
                                open=data['Open'],
                                high=data['High'],
                                low=data['Low'],
                                close=data['Close'],
                                name='Price'), row=1, col=1)

    # Ichimoku bileşenleri
    fig.add_trace(go.Scatter(x=data.index, y=data['Tenkan_sen'], 
                            name='Tenkan-sen', line=dict(color='royalblue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['Kijun_sen'], 
                            name='Kijun-sen', line=dict(color='red')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['Senkou_Span_A'],
                            name='Senkou Span A',
                            line=dict(color='rgba(76,175,80,0.3)')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['Senkou_Span_B'],
                            fill='tonexty',
                            name='Senkou Span B',
                            line=dict(color='rgba(255,82,82,0.3)')), row=1, col=1)

    # SAR noktaları
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['SAR'],
        mode='markers',
        name='Parabolic SAR',
        marker=dict(
            symbol='diamond',
            size=4,
            color='black',
        )
    ), row=1, col=1)

    # Alım-satım noktaları
    buy_signals = data[data['Signal'] == 1].index
    sell_signals = data[data['Signal'] == -1].index

    # Alım noktaları (yeşil üçgen)
    fig.add_trace(go.Scatter(
        x=buy_signals,
        y=data.loc[buy_signals, 'Low'] * 0.99,
        mode='markers',
        name='Buy Signal',
        marker=dict(
            symbol='triangle-up',
            size=15,
            color='green',
        )
    ), row=1, col=1)

    # Satım noktaları (kırmızı üçgen)
    fig.add_trace(go.Scatter(
        x=sell_signals,
        y=data.loc[sell_signals, 'High'] * 1.01,
        mode='markers',
        name='Sell Signal',
        marker=dict(
            symbol='triangle-down',
            size=15,
            color='red',
        )
    ), row=1, col=1)

    # Portfolio değeri
    fig.add_trace(go.Scatter(x=data.index, y=data['Portfolio_Value'],
                            name='Portfolio Value',
                            line=dict(color='purple')), row=2, col=1)

    # RSI
    fig.add_trace(go.Scatter(x=data.index, y=data['RSI'],
                            name='RSI',
                            line=dict(color='blue')), row=3, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

    # Pozisyonlar
    fig.add_trace(go.Scatter(x=data.index, y=data['Position'],
                            name='Position',
                            line=dict(color='orange')), row=4, col=1)

    # Grafik düzeni
    fig.update_layout(
        title='Ichimoku Cloud Strategy Backtest Results',
        yaxis_title='Price',
        template='plotly_white',
        height=1200,
        xaxis_rangeslider_visible=False,
        showlegend=True
    )

    # Her subplot için y ekseni başlıkları
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Portfolio Value ($)", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=3, col=1)
    fig.update_yaxes(title_text="Position Size", row=4, col=1)

    fig.show()

In [7]:
def print_trade_summary(data):
    """
    Alım-satım işlemlerinin özetini yazdırır
    """
    buy_signals = data[data['Signal'] == 1]
    sell_signals = data[data['Signal'] == -1]
    
    print("\nTrade Summary:")
    print("-" * 50)
    print(f"Total number of trades: {len(buy_signals)}")
    print("\nBuy Signals:")
    for date in buy_signals.index:
        print(f"Buy  @ {date.date()} - Price: ${data.loc[date, 'Close']:.2f}")
    
    print("\nSell Signals:")
    for date in sell_signals.index:
        print(f"Sell @ {date.date()} - Price: ${data.loc[date, 'Close']:.2f}")

In [8]:
if __name__ == '__main__':
    # Veri al
    data = yf.download("BTC-USD", start="2022-01-01")
    
    # Göstergeleri hesapla
    data = calculate_indicators(data)
    
    # Backtest'i çalıştır
    data, results = run_backtest(data)
    
    print_trade_summary(data)
    # Sonuçları görselleştir
    plot_strategy(data)

[*********************100%%**********************]  1 of 1 completed
Starting Portfolio Value: 100000.00
2022-06-04 BUY CREATE @ 29832.91
Final Portfolio Value: 147976.84
ROI: 47.98%

Trade Summary:
--------------------------------------------------
Total number of trades: 1

Buy Signals:
Buy  @ 2022-03-21 - Price: $41078.00

Sell Signals:
